# Comprehensive A/B Testing Analysis: Facebook vs. AdWords

## 📘 Introduction
Welcome to this data analysis notebook! In this project, we will compare the performance of two advertising campaigns: **Facebook** and **AdWords** (Google Ads).

### 🎯 Objective
Our goal is to determine which platform yields better results in terms of **Clicks**, **Conversions**, and overall **Cost Efficiency**.

### 🛠 Workflow
1. **Data Loading & Inspection**: Understanding our dataset.
2. **Data Cleaning**: Formatting the data for analysis.
3. **Exploratory Data Analysis (EDA)**: Visualizing trends and distributions.
4. **Hypothesis Testing**: Statistically proving which campaign is better.
5. **Regression Analysis**: Predicting future conversions.
6. **Time Series Analysis**: Identifying weekly and monthly patterns.

## 1. Import Libraries
We start by importing the necessary tools:
- **Pandas**: For handling tabular data (the dataset).
- **Matplotlib / Seaborn**: For creating charts and graphs.
- **Scikit-learn**: For building machine learning models (Regression).
- **Statsmodels/Scipy**: For statistical testing (T-Test).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import coint
from scipy import stats
import warnings

# Suppress warnings to keep the output clean
warnings.filterwarnings('ignore')

## 2. Load Data & Initial Inspection
Here we load the CSV file into a proper DataFrame. We will inspect:
- **.head()**: The first 5 rows to see what the data looks like.
- **.info()**: To check data types (strings vs numbers) and missing values.
- **.describe()**: For a quick statistical summary (mean, max, min).

In [ ]:
# Load the dataset
df = pd.read_csv('/Users/piyushpanthi/Documents/Data Analytics/A:B Testing/A_B_testing_dataset.csv') 

# Display first few rows to understand structure
print("--- First 5 Rows ---")
display(df.head())

# Check shape of data (Rows, Columns)
print("\n--- Dataset Shape ---")
print(df.shape)

# Check data types and information regarding missing values
print("\n--- Data Info ---")
print(df.info())

# Statistical summary of numerical columns
print("\n--- Statistical Summary ---")
display(df.describe())

## 3. Data Cleaning
Real-world data is often "messy". 
- Columns like `Cost` or `CTR` might contain symbols like `$` or `%`.
- Python reads these as **Strings** (text), but we need them as **Floats** (numbers) for calculation.
- We also need to convert the `Date` column to a datetime object for time-based analysis.

In [ ]:
# Convert 'Date' column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# Function to remove '$' and ',' from currency columns
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', ''))
    return x

# Function to remove '%' from percentage columns
def clean_percentage(x):
    if isinstance(x, str):
        return float(x.replace('%', ''))
    return x

# Apply cleaning to Facebook columns
df['Facebook_Cost_per_Ad'] = df['Facebook_Cost_per_Ad'].apply(clean_currency)
df['Facebook_CTR'] = df['Facebook_CTR'].apply(clean_percentage)
df['Facebook_Conversion_Rate'] = df['Facebook_Conversion_Rate'].apply(clean_percentage)
df['Facebook_CPC'] = df['Facebook_CPC'].apply(clean_currency)

# Apply cleaning to AdWords columns
df['AdWords_Cost_per_Ad'] = df['AdWords_Cost_per_Ad'].apply(clean_currency)
df['AdWords_CTR'] = df['AdWords_CTR'].apply(clean_percentage)
df['AdWords_Conversion_Rate'] = df['AdWords_Conversion_Rate'].apply(clean_percentage)
df['AdWords_CPC'] = df['AdWords_CPC'].apply(clean_currency)

# Verify that data types are now correct (float64 or datetime)
print("--- Data Types After Cleaning ---")
print(df.dtypes)

## 4. Exploratory Data Analysis (EDA)
### Distributions
We use **Histograms** to see the frequency distribution of our data.
- **Normal Distribution (Bell Curve)**: Data is symmetric around the mean.
- **Skewed**: Data leans to one side.

This helps us decide which statistical tests to use later.

In [ ]:
# Comparing distribution of Clicks and Conversions
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Histogram for Clicks
sns.histplot(df['Facebook_Clicks'], kde=True, color='blue', label='Facebook', ax=ax[0])
sns.histplot(df['AdWords_Clicks'], kde=True, color='orange', label='AdWords', ax=ax[0])
ax[0].set_title('Distribution of Clicks')
ax[0].legend()

# Histogram for Conversions
sns.histplot(df['Facebook_Conversions'], kde=True, color='blue', label='Facebook', ax=ax[1])
sns.histplot(df['AdWords_Conversions'], kde=True, color='orange', label='AdWords', ax=ax[1])
ax[1].set_title('Distribution of Conversions')
ax[1].legend()

plt.show()

### Boxplots for Comparison
Let's look at the "spread" of the data more clearly. Boxplots show us:
- The **Median** (middle value).
- The **Interquartile Range** (where the middle 50% of data lies).
- **Outliers** (dots outside the whiskers).

In [ ]:
# Prepare data for boxplot visualization
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

sns.boxplot(data=df[['Facebook_Clicks', 'AdWords_Clicks']], ax=ax[0], palette=['blue', 'orange'])
ax[0].set_title('Boxplot: Clicks Comparison')

sns.boxplot(data=df[['Facebook_Conversions', 'AdWords_Conversions']], ax=ax[1], palette=['blue', 'orange'])
ax[1].set_title('Boxplot: Conversions Comparison')

plt.show()

## 5. Correlation Analysis
We want to know: **Does getting more Clicks actually lead to more Conversions?**
- **Correlation = 1**: Perfect positive relationship (More clicks = More conversions).
- **Correlation = 0**: No relationship.
- **Correlation = -1**: Negative relationship.

In [ ]:
# Scatter plot to check relationship between Clicks and Conversions
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Facebook_Clicks', y='Facebook_Conversions', data=df, label='Facebook', color='blue')
sns.scatterplot(x='AdWords_Clicks', y='AdWords_Conversions', data=df, label='AdWords', color='orange')
plt.title('Relationship: Clicks vs Conversions')
plt.xlabel('Clicks')
plt.ylabel('Conversions')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# Calculate Correlation Coefficient
fb_corr = df['Facebook_Clicks'].corr(df['Facebook_Conversions'])
adwords_corr = df['AdWords_Clicks'].corr(df['AdWords_Conversions'])

print(f"Correlation - Facebook (Clicks -> Conversions): {fb_corr:.4f}")
print(f"Correlation - AdWords  (Clicks -> Conversions): {adwords_corr:.4f}")

## 6. A/B Testing (Hypothesis Testing)
Now we determine if the difference we see is **statistically significant** or just due to chance.

- **Null Hypothesis (H0)**: There is NO difference between Facebook and AdWords.
- **Alternative Hypothesis (H1)**: There IS a significant difference.

We use a **T-Test**. If the **P-Value < 0.05**, we reject the Null Hypothesis (meaning the difference is real).

In [ ]:
# Calculate mean conversions for context
print("Mean Facebook Conversions:", df['Facebook_Conversions'].mean())
print("Mean AdWords Conversions:", df['AdWords_Conversions'].mean())

# Perform T-Test (Independent samples)
t_stat, p_val = stats.ttest_ind(df['Facebook_Conversions'], df['AdWords_Conversions'])

print(f"\nT-statistic: {t_stat}")
print(f"P-value: {p_val}")

print("\n--- Conclusion ---")
if p_val < 0.05:
    print("✅ Result: Reject Null Hypothesis. There is a SIGNIFICANT difference between campaigns.")
else:
    print("❌ Result: Fail to reject Null Hypothesis. No significant difference found.")

## 7. Regression Analysis (Forecasting)
Since Facebook showed a strong correlation, can we **predict** conversions based on future clicks?
We train a **Linear Regression** model:
- **X (Independent Variable)**: Clicks
- **y (Dependent Variable)**: Conversions

We evaluate the model using R-squared (Accuracy) and MSE (Error).

In [ ]:
# Preparing Data for Regression (Using Facebook Data as it usually performs better in this dataset)
X = df[['Facebook_Clicks']] # Independent Variable
y = df['Facebook_Conversions'] # Dependent Variable

# Initialize and Fit Model
lm = LinearRegression()
lm.fit(X, y)

# Make Predictions across the existing data
predictions = lm.predict(X)

# Model Evaluation
r2 = r2_score(y, predictions)
mse = mean_squared_error(y, predictions)

print(f"R-squared Score (Accuracy): {r2:.4f}")
print(f"Mean Squared Error: {mse:.4f}")

# Plotting Regression Line
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Actual Data', alpha=0.6)
plt.plot(X, predictions, color='red', linewidth=3, label='Regression Line (Prediction)')
plt.xlabel('Clicks')
plt.ylabel('Conversions')
plt.title('Linear Regression: Predicting Facebook Conversions')
plt.legend()
plt.show()

# Interactive Prediction Example
clicks_to_predict = [[50], [80], [100]]
predicted_conversions = lm.predict(clicks_to_predict)

print("--- Future Predictions ---")
for i, clicks in enumerate(clicks_to_predict):
    print(f"If we get {clicks[0]} clicks, we predict {predicted_conversions[i]:.2f} conversions.")

## 8. Time Series Analysis
Finally, we analyze trends over time.
1. **Weekly Trends**: Which day of the week performs best? (Useful for ad scheduling)
2. **Monthly Trends**: Seasonality effects over the year.

In [ ]:
# Extract Month and Weekday names from Date
df['Month'] = df['Date'].dt.month_name()
df['Day_of_Week'] = df['Date'].dt.day_name()

# 1. Weekly Analysis
weekly_conv = df.groupby('Day_of_Week')[['Facebook_Conversions', 'AdWords_Conversions']].sum()
# Reorder days for correct plotting
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_conv = weekly_conv.reindex(days_order)

weekly_conv.plot(kind='bar', figsize=(12, 6))
plt.title('Total Weekly Conversions: Best Days to Advertise')
plt.xlabel('Day of Week')
plt.ylabel('Total Conversions')
plt.grid(axis='y', alpha=0.3)
plt.show()

# 2. Monthly Analysis
monthly_conv = df.groupby('Month')[['Facebook_Conversions', 'AdWords_Conversions']].sum()
# Reorder months
months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
monthly_conv = monthly_conv.reindex(months_order)

monthly_conv.plot(figsize=(12, 6), marker='o')
plt.title('Monthly Conversion Trends: Seasonality')
plt.xlabel('Month')
plt.ylabel('Total Conversions')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()